# Written Exercise 13.

The starter code evaluates the debiasing schemes on group unbalanced data in that there are
more males in the dataset than females. What impact does the dataset balance have on the debiasing method
and its results? As part of your answer experiment with debiasing balanced datasets and datasets even less
balanced than the provided one. Report the results. Pre-processing for balance is already included in the
starter code’s main section.

In [38]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow.data import Dataset
from tensorflow.errors import OutOfRangeError
from tensorflow.keras.layers import Input, Dense, Concatenate, Activation
from tensorflow.keras.models import Model

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import copy

from tqdm import tqdm as _tqdm

from hw5_part2 import *

### Step 1. Make Method of Obtaining Datasets with Different Balance

In [32]:
def norme(d):
    d = d.astype('float32')
    d -= d.min(axis=0)
    d -= d.max(axis=0) * 0.5
    d /= d.max(axis=0)
    return d

def sample(d, g0, g1, n, balance=False):
    if balance:
        return np.vstack([d[g0][0:n//2], d[g1][0:n//2]])
    else:
        return d[0:n]
    
def balance_sample(X,y,z,n,balance_ratio):
    """
    Take dataset and return proportion of balance
    """
    
    zi = z.reshape((len(z)))
    g0 = zi == 0.0
    g1 = zi == 1.0
    
    num_g1 = int(np.floor(n * balance_ratio))
    num_g0 = int(np.floor(n * (1-balance_ratio)))
    
    X_g1 = X[g1][0:num_g1]
    X_g0 = X[g0][0:num_g0]
    X_r = np.append(X_g1,X_g0,axis=0)
    
    y_g1 = y[g1][0:num_g1]
    y_g0 = y[g0][0:num_g0]
    y_r = np.append(y_g1,y_g0,axis=0)
    
    z_g1 = z[g1][0:num_g1]
    z_g0 = z[g0][0:num_g0]
    z_r = np.append(z_g1,z_g0,axis=0)
    
    return X_r, y_r, z_r

### Step 2. Sweep different proportions and make datasets for fix

In [39]:
# Load Dataset
temp = np.load("adult.npz")

X, y, z = norme(temp['X']), \
    temp['y'].astype('float32'), \
    temp['z'].astype('float32')

prop_list = [0.9, 0.8, 0.6, 0.5]
X_r = []
y_r = []
z_r = []

for i in range(len(prop_list)):
    X_temp, y_temp, z_temp = balance_sample(X,y,z,10000,prop_list[i])
    X_r += [copy.copy(X_temp)]
    y_r += [copy.copy(y_temp)]
    z_r += [copy.copy(z_temp)]

### Step 3. Run Adversity Training to get demographic parity and equality of opportunity fixes

In [42]:
def make_adult_classifier():
    c_in = Input((X.shape[1],))
    c_inter = Dense(32, activation="relu")(c_in)
    c_inter = Dense(32, activation="relu")(c_inter)
    c_inter = Dense(32, activation="relu")(c_inter)
    c_out = Dense(1, activation='sigmoid')(c_inter)

    return Model(c_in, c_out)

In [44]:
orig_dem = []
orig_eq  = []

dem_dem  = []
dem_eq   = []

eq_dem   = []
eq_eq    = []

ep = 5

for i in range(len(prop_list)):
    print("Proportion:",prop_list[i])
    
    # Predict Original
    print("Original Model...")
    
    c_orig = make_adult_classifier()
    c_orig.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    c_orig.fit(X_r[i], y_r[i], epochs=ep, batch_size=16)

    y_pred = (c_orig.predict(X_r[i]) > 0.5) * 1
    orig_dem += [evaluate_dem_parity(y_pred, y_r[i], z_r[i])]
    orig_eq += [evaluate_eq_op(y_pred, y_r[i], z_r[i])]
    
    
    # Predict demographic fix
    print("Demographic Model...")
    
    c_dem_par = AdversarialFairModel(make_adult_classifier())
    c_dem_par.train_dem_parity(
        X_r[i], y_r[i], z_r[i], epochs=ep, batch_size=16,
        display_progress=False
    )

    y_pred = (c_dem_par.predict(X_r[i]) > 0.5) * 1
    dem_dem += [evaluate_dem_parity(y_pred, y_r[i], z_r[i])]
    dem_eq += [evaluate_eq_op(y_pred, y_r[i], z_r[i])]
    
    # Predict equality fix
    print("Equality Model...")
    
    c_eq_op = AdversarialFairModel(make_adult_classifier())
    c_eq_op.train_eq_op(
        X_r[i], y_r[i], z_r[i], epochs=ep, batch_size=16,
        display_progress=False
    )
    
    y_pred = (c_eq_op.predict(X_r[i]) > 0.5) * 1
    eq_dem += [evaluate_dem_parity(y_pred, y_r[i], z_r[i])]
    eq_eq += [evaluate_eq_op(y_pred, y_r[i], z_r[i])]

Proportion: 0.9
Original Model...
Train on 9999 samples
Epoch 1/5
9999/9999 [==============================] - 1s 139us/sample - loss: 0.4610 - acc: 0.7730
Epoch 2/5
9999/9999 [==============================] - 1s 111us/sample - loss: 0.4139 - acc: 0.7958
Epoch 3/5
9999/9999 [==============================] - 1s 107us/sample - loss: 0.4041 - acc: 0.8013
Epoch 4/5
9999/9999 [==============================] - 1s 106us/sample - loss: 0.3985 - acc: 0.8048
Epoch 5/5
9999/9999 [==============================] - 1s 109us/sample - loss: 0.3952 - acc: 0.8050
Demographic Model...

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


bootstrapping classifier:   0%|          | 0/4 [00:00<?, ?epoch/s]

classifier:   0%|          | 0/624 [00:00<?, ?batch/s]

classifier:   0%|          | 1/624 [00:00<08:14,  1.26batch/s]

classifier:  10%|█         | 63/624 [00:00<05:11,  1.80batch/s]

classifier:  20%|█▉        | 124/624 [00:00<03:14,  2.57batch/s]

classifier:  28%|██▊       | 174/624 [00:01<02:03,  3.66batch/s]

classifier:  34%|███▍      | 214/624 [00:01<01:18,  5.20batch/s]

classifier:  40%|████      | 252/624 [00:01<00:50,  7.39batch/s]

classifier:  51%|█████     | 318/624 [00:01<00:29, 10.50batch/s]

classifier:  60%|█████▉    | 374/624 [00:01<00:16, 14.88batch/s]

classifier:  68%|██████▊   | 423/624 [00:01<00:09, 20.87batch/s]

classifier:  75%|███████▌  | 468/624 [00:01<00:05, 29.06batch/s]

classifier:  85%|████████▌ | 531/624 [00:01<00:02, 40.71batch/s]

classifier:  93%|█████████▎| 579/624 [00:02<00:00, 55.41batch/s]

classifier: 100%|█████████▉| 623/624 [00:02<00:00, 74.34batch/s]

                     

adversary:  86%|████████▌ | 534/624 [00:00<00:00, 1326.28batch/s]
                                                                 
adversary:  85%|████████▍ | 530/624 [00:00<00:00, 1333.88batch/s]
                                                                 
adversary:   0%|          | 0/624 [00:00<?, ?batch/s]
                                                     

Equality Model...



training:   0%|          | 0/5 [00:00<?, ?epoch/s]

bootstrapping classifier:   0%|          | 0/4 [00:00<?, ?epoch/s]


classifier:   0%|          | 0/624 [00:00<?, ?batch/s]


classifier:   0%|          | 1/624 [00:00<07:02,  1.47batch/s]


classifier:  15%|█▍        | 91/624 [00:00<04:13,  2.10batch/s]


classifier:  28%|██▊       | 174/624 [00:00<02:29,  3.00batch/s]


classifier:  40%|███▉      | 249/624 [00:00<01:27,  4.28batch/s]


classifier:  52%|█████▏    | 324/624 [00:01<00:49,  6.10batch/s]


classifier:  67%|██████▋   | 415/624 [00:01<00:24,  8.69batch/s]


classifier:  81%|████████  | 506/624 [00:01<00:09, 12.37batch/s]


classifier:  96%|█████████▌| 599/624 [00:01<00:01, 17.57batch/s]


                                                                

bootstrapping classifier:  25%|██▌       | 1/4 [00:01<00:04,  1.42s/epoch]


classifier:   0%|          | 0/624 [00:00<?, ?batch/s]


classifier:  13%|█▎        | 84/624 [00:00<00:00, 832.89batch/s]


classifier:  28%|██▊ 

...







training:   0%|          | 0/5 [00:00<?, ?epoch/s]






bootstrapping classifier:   0%|          | 0/4 [00:00<?, ?epoch/s]







classifier:   0%|          | 0/625 [00:00<?, ?batch/s]







classifier:   0%|          | 1/625 [00:00<10:23,  1.00batch/s]







classifier:  13%|█▎        | 81/625 [00:01<06:20,  1.43batch/s]







classifier:  26%|██▌       | 164/625 [00:01<03:45,  2.04batch/s]







classifier:  40%|███▉      | 247/625 [00:01<02:09,  2.91batch/s]







classifier:  54%|█████▍    | 336/625 [00:01<01:09,  4.16batch/s]







classifier:  68%|██████▊   | 422/625 [00:01<00:34,  5.92batch/s]







classifier:  81%|████████  | 506/625 [00:01<00:14,  8.44batch/s]







classifier:  95%|█████████▍| 593/625 [00:01<00:02, 12.00batch/s]







                                                                






bootstrapping classifier:  25%|██▌       | 1/4 [00:01<00:05,  1.75s/epoch]







classifier:   0%|          | 0/625 [00:00<?, ?batch/s]







classifier:  

classifier:  49%|████▊     | 304/625 [00:00<00:00, 730.35batch/s]






classifier:  62%|██████▏   | 385/625 [00:00<00:00, 752.01batch/s]






classifier:  75%|███████▍  | 467/625 [00:00<00:00, 770.14batch/s]






classifier:  88%|████████▊ | 549/625 [00:00<00:00, 783.51batch/s]






                                                                 





training:  60%|██████    | 3/5 [00:08<00:06,  3.35s/epoch]






adversary:   0%|          | 0/625 [00:00<?, ?batch/s]






adversary:  24%|██▎       | 147/625 [00:00<00:00, 1469.07batch/s]






adversary:  48%|████▊     | 298/625 [00:00<00:00, 1480.50batch/s]






adversary:  72%|███████▏  | 447/625 [00:00<00:00, 1481.48batch/s]






adversary:  95%|█████████▌| 594/625 [00:00<00:00, 1477.31batch/s]






                                                                 






classifier:   0%|          | 0/625 [00:00<?, ?batch/s]






classifier:  13%|█▎        | 84/625 [00:00<00:00, 835.65batch/s]






classifier:  26%|██▌    

Equality Model...









training:   0%|          | 0/5 [00:00<?, ?epoch/s]







bootstrapping classifier:   0%|          | 0/4 [00:00<?, ?epoch/s]








classifier:   0%|          | 0/625 [00:00<?, ?batch/s]








classifier:   0%|          | 1/625 [00:01<11:22,  1.09s/batch]








classifier:  13%|█▎        | 82/625 [00:01<06:55,  1.31batch/s]








classifier:  26%|██▌       | 164/625 [00:01<04:07,  1.86batch/s]








classifier:  39%|███▉      | 244/625 [00:01<02:23,  2.66batch/s]








classifier:  52%|█████▏    | 324/625 [00:01<01:19,  3.80batch/s]








classifier:  65%|██████▍   | 404/625 [00:01<00:40,  5.41batch/s]








classifier:  77%|███████▋  | 483/625 [00:01<00:18,  7.71batch/s]








classifier:  90%|████████▉ | 562/625 [00:01<00:05, 10.96batch/s]








                                                                







bootstrapping classifier:  25%|██▌       | 1/4 [00:01<00:05,  1.88s/epoch]








classifier:   0%|          | 0/625 [00:00<?, ?batch/s]








adversary:  67%|██████▋   | 421/625 [00:00<00:00, 1397.29batch/s]







adversary:  91%|█████████ | 566/625 [00:00<00:00, 1410.63batch/s]







                                                                 







classifier:   0%|          | 0/625 [00:00<?, ?batch/s]







classifier:  14%|█▎        | 85/625 [00:00<00:00, 842.82batch/s]







classifier:  27%|██▋       | 166/625 [00:00<00:00, 831.98batch/s]







classifier:  40%|███▉      | 247/625 [00:00<00:00, 824.73batch/s]







classifier:  52%|█████▏    | 327/625 [00:00<00:00, 816.52batch/s]







classifier:  65%|██████▌   | 407/625 [00:00<00:00, 809.91batch/s]







classifier:  78%|███████▊  | 487/625 [00:00<00:00, 805.49batch/s]







classifier:  91%|█████████ | 568/625 [00:00<00:00, 804.89batch/s]







                                                                 






training:  60%|██████    | 3/5 [00:08<00:07,  3.51s/epoch]







adversary:   0%|          | 0/625 [00:00<?, ?batch/s]







adversary:

### Step 4. Record Results In Table

In [52]:
eq_eq

[(0.07377049180327869, 0.5534407027818448),
 (0.11570247933884298, 0.5079034941763727),
 (0.4013452914798206, 0.5379501385041551),
 (0.40441176470588236, 0.4749833222148099)]